In [1]:
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [2]:
import sys
import os 
workdir = "/Users/mihaylov/research/qa-knowreader-long-docs"
sys.path.append("/Users/mihaylov/research/qa-knowreader-long-docs")
sys.path.append("/Users/mihaylov/research/qa-knowreader-long-docs/docqa")

os.chdir(workdir)

In [3]:
from docqa.commands.evaluate_qanet_semantic_flat import *
from docqa.data import *

Deprecated cache directory found (/Users/mihaylov/.allennlp/datasets).  Please remove this directory from your system to free up space.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


/Users/mihaylov/anaconda3/envs/docqa/lib/python3.6/site-packages/allennlp/commands/find_learning_rate.py:55: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/mihaylov/anaconda3/envs/docqa/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/mihaylov/anaconda3/envs/docqa/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mihaylov/anaconda3/envs/docqa/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mihaylov/anaconda3/envs/docqa/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/

In [4]:
#debug
model_path = "_trained_models/qanet_semantic_flat_concat_sdp_debug/model.tar.gz"

# model_path = "_trained_models/bidaf_qa_sumary_span_custom_multiview_eval/model.tar.gz"
# item_ids = "00936497f5884881f1df23f4834f6739552cee8b##016[15:30];00936497f5884881f1df23f4834f6739552cee8b##005[17:45];0029bdbe75423337b551e42bb31f9a102785376f##023"
# evaluation_data_file = "/Users/mihaylov/research/document-parsing-pipeline/tests/fixtures/data/narrativeqa/third_party/wikipedia/summaries-all.csv.parsed.jsonl.srl.jsonl.with_q_spans.jsonl.with_exp.with_sdp.json.train.2"
# output_file = "predictions_dev_jup.json"

model_path="_trained_models/summary_context_and_question_span_qanet_semantic_flat_concat_hs128_drop0.1_ml_srl_3verbs.json_19-04-30-17-52-03/model.tar.gz"
item_ids = "f4c9b2e062c118b0c15409e78b8e2a3c2ceadcd3##013[179:216];0d0bc01441afbec48663e3b38827e1529a287765##018[145:229]"
item_ids = "0d0bc01441afbec48663e3b38827e1529a287765##018[145:229]"
evaluation_data_file = "/Users/mihaylov/research/document-parsing-pipeline/tests/fixtures/data/narrativeqa/third_party/wikipedia/summaries-all.csv.parsed.jsonl.srl.jsonl.with_q_spans.jsonl.with_exp.with_sdp.json.test"
output_file = "predictions_test_vis.json"

args_ns = create_argparse_namespace(archive_file = workdir + "/"+model_path,
                              evaluation_data_file = evaluation_data_file,
                              output_file = "predictions_dev_jup.json",
                              item_ids = item_ids,
                              cuda_device = -1,
                              batch_size = 1,
                              output_attention = "True",
                              file_open_mode = "w",
                              overrides = "",
                              display_attention_matplot = "True",
                              )

# Display attentions


In [5]:
def draw(data, x, y, ax, x2=None, y1=None):
    seaborn.heatmap(data, 
                    xticklabels=x,
                    yticklabels=y, 
                    vmin=0.0, vmax=1.0, 
                    square=True, 
                    cbar=False, ax=ax)

    
def format_token(tkn_id, tkn, best_span, gold_span):
    prefix = ""
    suffix = ""
    if tkn_id == best_span[0]:
        prefix += "[" 
    if tkn_id == gold_span[0]:
        prefix += "{" 
    if tkn_id == gold_span[1]:
        suffix += "}" 
    if tkn_id == best_span[1]:
        suffix += "]" 
        
    return prefix + tkn + suffix
        
    
def display_attentions(tokens, meta, output, output_metadata, feat_id_to_feat_name, crop_range):
    modeling_layer_output = output_metadata["modeling_layer"]
    modeling_layer_iters = sorted(list(modeling_layer_output.keys()))
    
    #selected_span = output["best_span"].data[0].tolist()
    #gold_span = meta["token_spans"][0]
    #tokens = [format_token(tkn_id, tkn, selected_span, gold_span) for tkn_id, tkn in enumerate(metadata["passage_tokens"])]
    if crop_range is not None:
        print(crop_range)
        tokens = tokens[crop_range[0]: crop_range[1]]
        
    print(tokens)
    
    axs_head_iter = None
    for iter_key_id, iter_key in enumerate(modeling_layer_iters):
        iter_value = modeling_layer_output[iter_key]
        print("-" + iter_key)
        
        if iter_value is None:
            continue
            
        encoder_blocks = sorted(list(iter_value.keys()))
        encoder_blocks_cnt = len(encoder_blocks)
        for encoder_block_id, encoder_block_key in enumerate(encoder_blocks):
            print("--" + encoder_block_key)
            
            encoder_block_value = iter_value[encoder_block_key]
                                  
            attention_value = encoder_block_value["attention"]
            if isinstance(attention_value, list):
                attention_value = np.array(attention_value)
            
            semantic_views_q_value = encoder_block_value.get("semantic_views_q", None)
            if semantic_views_q_value is not None and not isinstance(semantic_views_q_value, list):
                semantic_views_q_value = semantic_views_q_value.tolist()
            
            semantic_views_sent_mask_value = encoder_block_value.get("semantic_views_sent_mask", None)
            if semantic_views_sent_mask_value is not None and not isinstance(semantic_views_sent_mask_value, list):
                semantic_views_sent_mask_value = semantic_views_sent_mask_value.tolist()
        
            num_heads = attention_value.shape[0]
            if axs_head_iter is None:
                fig, axs_head_iter = plt.subplots(num_heads, len(modeling_layer_iters) * encoder_blocks_cnt, 
                                                  figsize=(90 * encoder_blocks_cnt, 240))
                
            for head_id in range(num_heads):
                head_attention = attention_value[head_id]
                semantic_views_q_curr = None
                if semantic_views_q_value is not None:
                    semantic_views_q_curr = semantic_views_q_value[head_id]
                    
                if semantic_views_sent_mask_value is not None:
                    semantic_views_sent_mask_curr = semantic_views_sent_mask_value[head_id]
                
                #print("mask:")
                #print(semantic_views_sent_mask_curr)
                # normalize in [0,1.0]
                
                if crop_range is not None:
                    head_attention = head_attention[crop_range[0]:crop_range[1], crop_range[0]:crop_range[1]]
                
                    if semantic_views_q_curr is not None :
                        semantic_views_q_curr[crop_range[0]:crop_range[1]]
                
                print("head_attention{4}:max={0}, min={1}, mean={2}, std={3}".format(head_attention.max(), head_attention.min(), head_attention.mean(), head_attention.std(), head_id))
                head_attention = head_attention/max(0.001, head_attention.max())
                
                feats = []
                if semantic_views_q_curr is not None:
                    feats = [feat_id_to_feat_name.get(int(x), "") for x in semantic_views_q_curr]
                
                axs = axs_head_iter[head_id][iter_key_id * encoder_blocks_cnt + encoder_block_id]
                axs.set_title("iter {0}, block {1}, head {0}".format(iter_key_id, encoder_block_id, head_id))
                #fig, axs = plt.subplots(1, figsize=(10, 10))
                draw(head_attention, tokens, tokens if iter_key_id == 0 else feats, ax=axs)
                
#                 break
#             break
#         break      
    plt.show()
    
    

# Persist data - remove the break at the end!

In [6]:
import numpy as np

def replace_json_lists_with_shapes(json_item):
    new_json = {}
    for key, value in json_item.items():
        if isinstance(value, list):
            new_json[key] = str(np.array(value).shape)
        elif isinstance(value, dict):
            new_json[key] = replace_json_lists_with_shapes(value)
        elif isinstance(value, list):
            new_json[key] = str(value.shape)
        
    return new_json



In [7]:

# function for working with output of the model
def _persist_data_matplot(file_handle, 
                  metadata, 
                  model_output, 
                  id2label=None,
                  id_to_meta=None,
                  feat_id_to_feat_name=None,
                  display_attention_matplot=True) -> None:
    if metadata:
        batch_size = len(metadata)
        for index, meta in enumerate(metadata):
            res = {}
            item_id = meta.get("id", "n/a")
            res["id"] = item_id
            res["meta"] = meta
            # We persist model output which matches batch_size in length and is not a Variable
            for key, value in model_output.items():
                if key == "output_metadata":
                    crop_range = None
                    if id_to_meta is not None:
                        id_meta_curr = id_to_meta.get(item_id, None)
                        if id_meta_curr is not None:
                            crop_range = id_meta_curr["attention_range"]
        
                    sequence_len = model_output["span_start_logits"].shape[-1]
                    attentions_metadata = attentions_to_json(value, index, batch_size, sequence_len , crop_range)
                    attentions_shapes = replace_json_lists_with_shapes(attentions_metadata)
                    print(json.dumps(attentions_shapes, indent=4))
                    
                    print("Passage:")
                    print(meta["passage_tokens"])
                    print("Question:")
                    print(meta["question_tokens"])
                    print("Gold answer:")
                    gold_span = meta["token_spans"][0]
                    print(meta["passage_tokens"][gold_span[0]:gold_span[1]+1])
                    
                    print("Selected answer:")
                    print(meta["token_spans"][0])
                    
                    print("feat_id_to_feat_name:")
                    print(feat_id_to_feat_name)
                    display_attentions(meta["passage_tokens"], meta, model_output, attentions_metadata, feat_id_to_feat_name, crop_range)
                    # res["attentions_metadata"] = attentions_metadata
                    #print(attentions_metadata)

                curr_value = value
                if isinstance(value, torch.autograd.Variable) or isinstance(value, Tensor):
                    curr_value = value.data.tolist()

                if not isinstance(curr_value, torch.autograd.Variable) \
                        and isinstance(curr_value, list) \
                        and len(curr_value) == batch_size:
                    val = curr_value[index]
                    res[key] = val

            if "label_probs" in res and id2label is not None:
                labels_by_probs = sorted([[id2label[li], lp] for li, lp in enumerate(res["label_probs"])], key=lambda x:x[1], reverse=True)
                res["labels_by_prob"] = labels_by_probs
                res["label_predicted"] = labels_by_probs[0][0]

            file_handle.write(json.dumps(res))
            file_handle.write("\n")
            



In [8]:
# Evaluate evaluate_matplot
def evaluate_matplot(model: Model,
             instances: Iterable[Instance],
             data_iterator: DataIterator,
             output_file: str = None,
             file_mode="w",
             id_to_meta: Dict[str, Any] = {},
             feat_id_to_feat_name: Dict[int, str]= {},
             display_attention_matplot = False,
             ) -> Dict[str, Any]:
    model.eval()

    iterator = data_iterator(instances, num_epochs=1, shuffle=False)
    logger.info("Iterating over dataset")
    generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))
    with ExitStack() as stack:
        if output_file is None:
            file_handle = None
        else:
            file_handle = stack.enter_context(open(output_file, file_mode))

        for batch in generator_tqdm:
            model_output = model(**batch)
            metrics = model.get_metrics()
            if file_handle:
                id2label = model.vocab.get_index_to_token_vocabulary("labels")
                _persist_data_matplot(file_handle, batch.get("metadata"), model_output,
                              id2label=id2label,
                              id_to_meta=id_to_meta,
                              feat_id_to_feat_name=feat_id_to_feat_name,
                              display_attention_matplot=display_attention_matplot)
            description = ', '.join(["%s: %.2f" % (name, value) for name, value in metrics.items()]) + " ||"
            generator_tqdm.set_description(description)

    return model.get_metrics()

In [ ]:
metrics = evaluate_from_args(args_ns, func_eval=evaluate_matplot)


10557it [00:10, 1014.30it/s]
  0%|          | 0/1 [00:00<?, ?it/s]

{
    "modeling_layer": {
        "modeling_layer_iter_000": {
            "encoder_block_001": {
                "attention": "(8, 310, 310)",
                "semantic_views_q": "(8, 310)",
                "semantic_views_sent_mask": "(8, 310)",
                "mask": "(310,)"
            },
            "encoder_block_003": {
                "attention": "(8, 310, 310)",
                "semantic_views_q": "(8, 310)",
                "semantic_views_sent_mask": "(8, 310)",
                "mask": "(310,)"
            },
            "encoder_block_005": {
                "attention": "(8, 310, 310)",
                "semantic_views_q": "(8, 310)",
                "semantic_views_sent_mask": "(8, 310)",
                "mask": "(310,)"
            }
        },
        "modeling_layer_iter_001": {
            "encoder_block_001": {
                "attention": "(8, 310, 310)",
                "semantic_views_q": "(8, 310)",
                "semantic_views_sent_mask": "(8, 310)",
      

head_attention0:max=0.9160908460617065, min=0.0, mean=0.01176310381741844, std=0.04038773946334654
head_attention1:max=0.9072096347808838, min=0.0, mean=0.011767141297962684, std=0.05327492892801576
head_attention2:max=0.9542418122291565, min=0.0, mean=0.011763365009707994, std=0.04804298452283254
head_attention3:max=0.26755422353744507, min=1.89765833056299e-05, mean=0.003121904469626965, std=0.005214380307047593
head_attention4:max=0.15406565368175507, min=8.800298267885864e-09, mean=0.003769263949472674, std=0.007052620201022937
head_attention5:max=0.9923557043075562, min=1.037977238604797e-34, mean=0.004923546623688887, std=0.04516432229946667
head_attention6:max=0.11912782490253448, min=2.681115802261047e-06, mean=0.0034753076165427777, std=0.005740658281295934
head_attention7:max=0.2373305708169937, min=4.3457706415436546e-31, mean=0.002280713427758916, std=0.014233026924359762
--encoder_block_003
head_attention0:max=0.9848043918609619, min=0.0, mean=0.011768413392424942, std=0.0

In [ ]:

print("Results:")
print(json.dumps(metrics, indent=4))
